## Simulating Fourier Transform
<br>Author: Raghuttam Hombal<br>
<b>Using Julia for simulations and FFT coded in Python3</b>

In [ ]:
using PyCall
using Plots

In [ ]:
pushfirst!(PyVector(pyimport("sys")."path"), ".")
push!(pyimport("sys")."path", "/home/raghuttam/experimental/fft_trials/fft.py") #Please change the path to the absolute path in your device
fft = pyimport("fft") 

## Discrete Fourier Transform
It is a method to converting a signal from <b>Time Domain</b> to <b>Frequency Domain</b><br>More the details can be read [here](https://www.nxp.com/docs/en/application-note/AN4255.pdf)<br>
We have used the <b><i>Fast Fourier Transform</i></b>, as it is was the primary purpose of the project. The algorithm is implemented using Python (which can be found in the same repository). This module is called in the Julia console, and used.

Now when FFT is applied in Real-time, we take the sample and concatenate it to the last of the set of N-Points (here 64). Then, the Fourier Transform of the whole set is obtained and displayed in Real-time. The function below, does the same. It cuts first (oldest sample) and makes the list back to the standard size of DFT (here 64-point)

In [ ]:
function dftCalc()
    n = length(data)
    if n>64
        pop!(data)
    end
    return fft.FFT(data)
end


Trying the system with random inputs

In [ ]:
data = zeros(Int64,64)

anim = @animate for i in 1:150
    sample = rand((0,1,2,3))
    pushfirst!(data,sample)
    p1=plot(data,label="signal",ylim=(0,5))
    ft = abs.(dftCalc())
    ft = ft[Int(length(ft)/2):end]
    p2=scatter(ft,label="Fourier Transform",ylim=(0,100))
    plot(p1,p2,layout=2)
end
gif(anim,"dataVisualize.gif",fps=10)

### Using a Signal which is corrupted by noise.
A signal is generated of time duration 2 seconds, with sampling frequency 1kHz.
2 signals of 12f (Hz) and 8f (Hz) where f=10, are superposed. This Signal is introduced to a noise of a very small energy, whose Fourier Transform is demonstrated.<br>
This set of data given to the FFT calculator sample by sample and simulataneously, the FFT of the set is calculated. However, for the first few samples, the set is initially zero-padded to make it to 64-point, hence it takes some while to get to a stable situation

In [ ]:
duration,fS,f = 2,1000,10
t = LinRange(0,duration*fS,fS)/fS
signal = sin.(2*12*pi*f*t) + sin.(2*8*pi*f*t)
noise = rand(Float64,length(signal))/3
practical_signal = signal+noise
plot(t[1:101],practical_signal[1:101],title="Chunk of the Signal",label=false,xlabel="Time",ylabel="Amplitude")

In [ ]:
data = zeros(Float64,64)

anim = @animate for i in 1:length(practical_signal)
    sample = practical_signal[i]
    pushfirst!(data,sample)
    p1=plot(data,title="Signal",ylim=(-2,2),label=false,xlabel="Time")
    ft = abs.(dftCalc())
    ft = ft[Int(length(ft)/2):end]
    freqs = LinRange(0,fS/2,32)/duration
    p2=scatter(freqs,reverse(ft),title="Fourier Transform",ylim=(0,30),label=false,xlabel="Frequency")
    plot(p1,p2,layout=2)
end
gif(anim,"dataVisualize.gif",fps=30)

It can be seen that there are spikes at frequency 80Hz and 120Hz, which clearly tells us the presence of signal of that frequency in the input signal and the amplitude to the spike corresponds to the proportional contribution of that component to the input signal. Since input keeps changing at every instant, the equivalent Fourier transform is obtained at every instance and it keeps varing too.<br>Due to the noise in the input, It becomes very difficult to comprehend the Fourier Transform. This can be reduced, by reducing the noise in the input signal. This can be done by using <b>Moving Average Filter</b>


### Using Moving Average Filter (N=5)
This concept helps to reduce the noise, by averaging out the sample with the preceding inputs which reduced the small abrupt changes caused due to noise. The formula is given by
$$y[i] = \frac{1}{M} \sum_{j=0}^{M-1}x[i+j]$$
Choosing the N-point Moving Average Filter becomes critical and is also subjective, if N is very less, then the noise is retained in considerable amount, else if N is too large, then the structure of the signal is lost. <br>More of that can be read here: [MOVING AVERAGE FILTER](https://www.analog.com/media/en/technical-documentation/dsp-book/dsp_book_Ch15.pdf)

In [ ]:
function movingAvg(sample)
    return (sum(data[end-3:end])+sample)/5
end

In [ ]:
data = zeros(Float64,64)

anim = @animate for i in 1:length(practical_signal)
    sample = practical_signal[i]
    pushfirst!(data,movingAvg(sample))
    p1=plot(data,title="Signal",ylim=(-2,2))
    ft = abs.(dftCalc())
    ft = ft[Int(length(ft)/2):end]
    freqs = LinRange(0,fS/2,32)/duration
    p2=scatter(freqs,reverse(ft),title="Fourier Transform",ylim=(0,30))
    plot(p1,p2,layout=2)
end
gif(anim,"dataVisualize_mvgavg.gif",fps=30)

We can notice that the noise is considerably reduced, which makes the Fourier Transform more stable and easy to comprehend but the catch is that, The amplitude of the signal reduces.